In [1]:
import os
from pathlib import Path
from monai.apps.auto3dseg import AutoRunner
from monai.config import print_config
import shutil
import json
import warnings

print_config()

MONAI version: 1.4.0
Numpy version: 1.26.4
Pytorch version: 2.5.1+cu124
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 46a5272196a6c2590ca2589029eed8e4d56ff008
MONAI __file__: /home/<username>/.virtualenvs/monai/lib/python3.12/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: 5.4.0
Nibabel version: 5.3.2
scikit-image version: 0.24.0
scipy version: 1.14.1
Pillow version: 11.0.0
Tensorboard version: 2.18.0
gdown version: 5.2.0
TorchVision version: 0.20.1+cu124
tqdm version: 4.66.6
lmdb version: 1.5.1
psutil version: 6.1.0
pandas version: 2.2.3
einops version: 0.8.0
transformers version: 4.46.2
mlflow version: 2.17.2
pynrrd version: 1.1.1
clearml version: 1.16.5

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



You can edit these if you want

In [ ]:
model = "segresnet" #! edit this

description = \
"""
Fill this in with whatever information
you want about this training. it will be saved to a text file 
"""

Don't change anything below here.

Set all paths

In [ ]:
dataroot = Path("/media/smbshare/3Tpioneer_bids")
curr_dir = Path(os.getcwd())
work_dir = curr_dir / f"{model}_work_dir"

datalist_file_src = curr_dir /  "datalist.json"
datalist_file = work_dir / "datalist.json"

Scan datalist to make sure all the files exist

In [ ]:
with open(datalist_file_src, 'r') as f:
    datalist = json.load(f)

testing = datalist['testing']
training = datalist['training']

for item in training:
    if not os.path.exists(item['image']):
        raise FileNotFoundError(f"Cannot find image {item['image']}, double check training data in datalist.json")
    if not os.path.exists(item['label']):
        raise FileNotFoundError(f"Cannot find label {item['label']}, double check training data in datalist file")
    
for item in testing:
    if not os.path.exists(item['image']):
        raise FileNotFoundError(f"Cannot find image {item['image']}, double check testing data in datalist.json")
    if not os.path.exists(item['label']):
        warnings.warn(f"Cannot find label {item['label']}, double check datalist file. Code will run, but you won't have a ground truth to compare")

Create the working directory for Auto3dSeg

In [ ]:
valid_models = ["segresnet", "swinunetr", "dints", "segresnet2d"]
if model not in valid_models:
    raise ValueError(f"The model provided is not valid. It should be one of: {", ".join(valid_models)}")

if not work_dir.exists():
    os.makedirs(work_dir)

if not datalist_file_src.exists():
    raise FileNotFoundError("datalist.json file cannot be found")

shutil.copyfile(datalist_file_src, datalist_file)

with open(work_dir / "info.txt", 'w') as f:
    f.write(description)

Setup training and run

In [ ]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=[model],
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

max_epochs = 250
train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

In [ ]:
runner.run()